In [ ]:
# Import libraries

import gw3RACE_functions


%matplotlib inline

In [ ]:
# Example code
df = pd.read_csv('output_short.tab', sep = '\t',
                 names = ['read_ID', 'chr', 'start_R1', 'stop_R1','strand_R1', 'gene_start', 'gene_stop',
                         'gene','coord_R2', 'cigar', 'seq_R2'])

df['RNA_type'] = df['gene'].apply(lambda x: test_RNA_type(x.lower()))
df['tail_fromcigar'] = df.apply(lambda kol: take_tail_fromcigar8(kol.strand_R1, kol.cigar,  kol.seq_R2), 
                                axis = 1)
df['tail_LENcigar'] = df['tail_fromcigar'].apply(lambda x: len(x))
df['tail_fromGREP'] = df.apply(lambda kol: grep_tail_edit_onlyfromSeq(kol.seq_R2), axis = 1)

#df = df.drop_duplicates()
df['tail_GreporCigar'] = df.apply(lambda kol: tail_fromGREPorCIGAR(kol.cigar, kol.tail_fromGREP, kol.tail_fromcigar),axis = 1)


                                                                                    
df['tail_from'] = df.apply(lambda kol: tail_fromGREPorCIGAR_description(kol.cigar,
                                                                        kol.tail_fromGREP,kol.tail_fromcigar),
                           axis = 1)
df['tail_len'] = df['tail_GreporCigar'].apply(lambda x: len(x))
#df = df[df['tail_len']>0]
df['tail_type'] = df.apply(lambda kol:   test_tail_cigargrep8(kol.tail_from, kol.strand_R1, kol.tail_GreporCigar),axis = 1)
  

df['stop_R2'] =  df.apply(lambda kol: stop_based_on_cigar(kol.cigar, kol.strand_R1 , kol.coord_R2),axis = 1)
df['distance_to_TES'] =  df.apply(lambda kol: distance_to_TES(kol.cigar,kol.strand_R1 ,kol.stop_R2,
                                                              kol.gene_start,kol.gene_stop),axis = 1)

df['gene_len'] = df['gene_stop']-df['gene_start']
df['rel_distance_to_TES'] = df['distance_to_TES']/df['gene_len']
df.to_csv('output.csv')
    

In [ ]:
df.head()